In [79]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import sklearn
import os
%matplotlib inline
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, LSTM, Bidirectional
import tensorflow as tf

In [6]:
# df = pd.read_csv("full_dataset_v1.csv")
# df = pd.read_csv("/Users/ishaanjavali/Documents/Science Fair/2020/Code/API/full_dataset_v1.csv")
df = pd.read_csv("/Users/ishaanjavali/Documents/Science Fair/2020/Code/API/full_dataset_good.csv")

In [7]:
df = df[df['severity'].isin(["medium", "small", "large", "very_large", "na"])]
df['severity'].value_counts()
df = shuffle(df)
df.reset_index(inplace=True, drop=True)

In [8]:
def getX():
    X = df.copy()

    X.drop(X.columns[[i for i in range(0, 151)]], axis = 1, inplace = True)
    # X.drop(X.columns[[i for i in range(20, 35)]], axis = 1, inplace = True)
    X["severity"] = df["severity"]
    X.drop(X.columns[[0]], axis = 1, inplace = True)

    X = X.dropna()
    for i in range(0, 8):
        del X['air' + str(i)]
        del X['temp' + str(i)]
        del X['humidity' + str(i)]
        
    X.drop(X.columns[[i for i in range(len(X.columns)-8, len(X.columns)-2)]], axis = 1, inplace = True)
    return X
X = getX()

In [90]:
def generate_labels(binary = False):
    global X
    X = getX()
    y = []
    idx_to_severity = ["na", "small", "medium", "large", "very_large"]
    for severity in X.severity:
        if not binary:
            y.append(idx_to_severity.index(severity))
        elif severity == "na":
            y.append(0)
        else:
            y.append(1)
    X.drop(X.columns[[-1,-2]], axis = 1, inplace = True)
    print(y.count(1))
    return y

In [95]:
binary = True
y = generate_labels(binary)
X.head()

4394


,precip7,wind7,precip6,wind6,precip5,wind5,precip4,wind4,precip3,wind3
0,0.1,20,0.3,13,0.0,15,0.4,14,0.1,17
1,0.0,16,0.0,16,0.0,14,0.5,15,0.0,13
2,0.0,10,0.0,9,0.0,10,0.0,11,0.0,10
3,0.0,18,0.0,16,0.0,19,1.0,18,1.2,15
4,0.0,18,0.0,18,0.0,13,0.2,17,0.0,19


In [96]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train[0]

array([ 6.55374823, -1.20714333,  0.93487281, -0.23135781,  1.65422362,
       -1.17180488,  1.06072668, -1.1515248 ,  0.79474075, -1.16530559])

In [104]:
y_train = np.array(y_train)
y_test = np.array(y_test)
# len(X_train) # of batches
# 5 timesteps (days 7 - 3)
# 2 features per timestep
X_train=X_train.reshape((len(X_train), 5, 2))
X_test=X_test.reshape((len(X_test), 5, 2))
print(X_train.shape)
X_train[0]

(7096, 5, 2)


array([[ 6.55374823, -1.20714333],
       [ 0.93487281, -0.23135781],
       [ 1.65422362, -1.17180488],
       [ 1.06072668, -1.1515248 ],
       [ 0.79474075, -1.16530559]])

In [106]:
model = Sequential()
model.add(Bidirectional(LSTM(11)))
model.add(Dense(1,activation='softmax'))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs = 10, batch_size = 20, verbose = 2)
# res = LSTM(11)
# out = res(X_train)
# print(out.shape)

Epoch 1/10
355/355 - 1s - loss: 0.5082
Epoch 2/10
355/355 - 1s - loss: 0.5082
Epoch 3/10
355/355 - 1s - loss: 0.5082
Epoch 4/10
355/355 - 1s - loss: 0.5082
Epoch 5/10
355/355 - 1s - loss: 0.5082
Epoch 6/10
355/355 - 1s - loss: 0.5082
Epoch 7/10
355/355 - 1s - loss: 0.5082
Epoch 8/10
355/355 - 1s - loss: 0.5082
Epoch 9/10
355/355 - 1s - loss: 0.5082
Epoch 10/10
355/355 - 1s - loss: 0.5082


In [123]:
# tx = X_test[0]
# ty = np.array([y_test[0]])
# tx = tx.reshape((1, 5, 2))
# model.evaluate(tx,ty)
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 819us/step - loss: 0.0000e+00


0.0